In [ ]:
import os
import sys

sys.path.append("..")

import ase.atoms
import ase.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.ensemble
import sklearn.metrics
import sklearn.model_selection

import config.paths as PATHS
import src.features.features_extractors as features_extractors
import src.features.input_parsers as input_parsers
import src.visualization as visualization

plt.style.use("seaborn-v0_8")

### Constans

In [ ]:
seed = 0xCAFFE

### Single particle loading for experiments: 

In [ ]:
# particle = ase.io.read(os.path.join('..', 'data', 'S_random_1.xyz'))
# particle

In [ ]:
# comments = comments_parser._load_lines_after_specified_one(
#     os.path.join("..", "data", "S_random_8000.xyz"),
#     "42\n"
# )
# y = pd.read_csv(os.path.join('..', 'data', 'S_random_8000.trans'), header=None)[0]

# Feature definition

In [ ]:
sulfur_idxs = [10, 31]

benzene1_idxs = [11, 14, 15, 16, 17, 20]
benzene2_idxs = [21, 24, 25, 26, 27, 30]

benzene1_plane_idxs = [14, 15, 16]
benzene2_plane_idxs = [25, 26, 27]

# Apply to all particles:

In [ ]:
df = input_parsers.read_raw_data(
    PATHS.PARTICLES_FILE, PATHS.TRANSPORT_FILE, PATHS.FEATURES_CACHE
)
df["y"] = np.log(df["y"])  # ToDo: log dodany do y - refactor it
df.head()

In [ ]:
features_extractors.add_benzene_dst_feature(df, benzene1_idxs, benzene2_idxs)
features_extractors.add_benzene_cossq_feature(df, [11, 16, 17], [24, 27, 25])
features_extractors.add_dst_feature(df, 32, 9)  # AuAu33_10
features_extractors.add_dst_feature(df, 20, 21)  # CC21_22
features_extractors.add_dst_feature(df, 31, 32)  # SAu32_33
features_extractors.add_dst_feature(df, 32, 33)  # Au-Au (33-34)
features_extractors.add_dst_feature(df, 30, 31)  # C-S (31-32)
features_extractors.add_dst_feature(df, 27, 29)  # C-H (28-30)
features_extractors.add_dst_feature(df, 15, 17)  # C-C (16-18)

features_extractors.add_ang_feature(df, 32, 31, 30)  # Au_S_C_idxs_1
features_extractors.add_ang_feature(df, 9, 10, 11)  # Au_S_C_idxs_2
features_extractors.add_ang_feature(df, 33, 32, 31)  # Au_Au_S_idxs_1
features_extractors.add_ang_feature(df, 32, 21, 9)  # Au_C_Au_idxs
features_extractors.add_ang_feature(df, 20, 22, 23)  # H_C_H_idxs
features_extractors.add_ang_feature(df, 21, 24, 26)  # C-C-C (np. 22-25-27)
features_extractors.add_ang_feature(df, 24, 25, 27)  # H-C-C (np. 24-26-28)
features_extractors.add_ang_feature(df, 27, 30, 31)  # C-C-S (np. 28-31-32)
features_extractors.add_ang_feature(df, 31, 33, 35)  # Au-Au-Au (33-34-36)

# kąt torsyjny Au-S-C-C (np. 33-32-31-27)
features_extractors.add_dih_feature(df, 32, 31, 30, 26)
# kąt torsyjny Au-Au-S-C (np. 8-10-11-12)
features_extractors.add_dih_feature(df, 7, 9, 10, 11)

comments_df = input_parsers.get_comments_df(PATHS.PARTICLES_FILE)
df["fermi_energy"] = comments_df["fermi_energy"]
df["total_energy"] = comments_df["total_energy"]
df["HOMO_LUMO_diff"] = comments_df["energy_level_130"] - comments_df["energy_level_131"]
# df["electron_state_143"] = comments_df["electron_state_143"]
# df["electron_state_144"] = comments_df["electron_state_144"]
df.head()

In [ ]:
# ToDo: zmienic korelację na rank - tao-kendala rosse-...

corr_matrix = df.drop("obj", axis=1).corr(numeric_only=False)
fig, ax = plt.subplots()
fig.set_size_inches(15, 15)
visualization.draw_correlation_matrix(corr_matrix, ax)
plt.show()
#  ToDo: cossingus kąta benzenu do kwadratu jako predyktor

In [ ]:
corr_treshold = 0.00
restricted_features = corr_matrix[corr_matrix["y"].abs() >= corr_treshold].index[1:]

# Trening

In [ ]:
y = df["y"]
X = df[restricted_features]

y.shape, X.shape

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.25, random_state=seed
)
for x in [X_train, X_test, y_train, y_test]:
    print(x.shape)

In [ ]:
clf = sklearn.ensemble.GradientBoostingRegressor(
    # learning_rate=0.25, max_depth=3, n_estimators=50, random_state=seed
    **dict(
        {
            "ccp_alpha": 0,
            "learning_rate": 0.2,  # 0.25
            "max_depth": 4,
            "max_features": "sqrt",
            "n_estimators": 30,
            "random_state": seed,
            "subsample": 0.8,  # 1
        }
    )
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

eval_y_test, eval_y_pred = np.exp(y_test), np.exp(y_pred)
eval_y_train, eval_y_train_pred = np.exp(y_train), np.exp(y_train_pred)

mse = sklearn.metrics.mean_squared_error(eval_y_test, eval_y_pred)
rmse = sklearn.metrics.mean_squared_error(eval_y_test, eval_y_pred, squared=False)
mae = sklearn.metrics.mean_absolute_error(eval_y_test, eval_y_pred)
mape = sklearn.metrics.mean_absolute_percentage_error(eval_y_test, eval_y_pred)

train_mse = sklearn.metrics.mean_squared_error(eval_y_train, eval_y_train_pred)
train_rmse = sklearn.metrics.mean_squared_error(
    eval_y_train, eval_y_train_pred, squared=False
)
train_mae = sklearn.metrics.mean_absolute_error(eval_y_train, eval_y_train_pred)
train_mape = sklearn.metrics.mean_absolute_percentage_error(
    eval_y_train, eval_y_train_pred
)

print(
    "Metric \tTest \tTrain",
    "#" * 22,
    "MSE:\t{:6.4f}\t{:6.4f}".format(mse, train_mse),
    "RMSE:\t{:6.4f}\t{:6.4f}".format(rmse, train_rmse),
    "MAE:\t{:6.4f}\t{:6.4f}".format(mae, train_mae),
    "MAPE:\t{:6.4f}\t{:6.4f}".format(mape, train_mape),
    sep="\n",
)
# MAPE:	0.4837	0.3913 default

In [ ]:
# plt.bar(range(len(y_test)), sorted(list(y_test - y_pred)))
results_df = pd.DataFrame(
    {"y_test": eval_y_test, "y_pred": eval_y_pred, "diff": eval_y_test - eval_y_pred}
)
print(results_df["diff"].abs().std())
print(y_pred.std(), y_test.std())
results_df.sort_values("y_test")["diff"].plot.bar()

In [ ]:
results_df.sort_values("y_test")[["y_test", "y_pred"]].plot.scatter("y_test", "y_pred")

In [ ]:
y_test.mean()

In [ ]:
abs_correlations = (
    corr_matrix.loc["y", corr_matrix["y"].abs() >= corr_treshold]
    .abs()
    .rename("abs_corelations")
)
features_importance = pd.Series(
    clf.feature_importances_, index=restricted_features
).rename("feature_importance")

importance_df = pd.merge(
    features_importance, abs_correlations, left_index=True, right_index=True
)
importance_df.sort_values("feature_importance").plot.bar()
plt.show()

# Grid Search for complex analise

In [ ]:
# from sklearn.metrics import make_scorer
# from sklearn.model_selection import GridSearchCV


# def calculate_mape(y_true, y_pred):
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


# mape_scorer = make_scorer(calculate_mape, greater_is_better=False)

# # create a GradientBoostingRegressor object
# clf = sklearn.ensemble.GradientBoostingRegressor()

# # define the hyperparameter grid to search over
# param_grid = {
#     "learning_rate": [0.3, 0.25, 0.2],
#     "n_estimators": [30],
#     "max_depth": [3, 4],
#     "subsample": [0.75, 1],
#     "n_iter_no_change": [3, None],
#     "ccp_alpha": [0, 0.001, 0.1, 0.5],
#     "max_features": [1.0, "sqrt", "log2", None],
#     "random_state": [seed],
# }

# # define the grid search object
# grid_search = GridSearchCV(
#     clf,
#     param_grid=param_grid,
#     cv=5,
#     scoring=mape_scorer,  #'neg_mean_squared_error',
#     n_jobs=-1,
# )

# # fit the grid search object to the data
# grid_search.fit(X_train, y_train)

# # print the best hyperparameters and the corresponding mean squared error
# print("Best hyperparameters: ", grid_search.best_params_)
# print("Best MSE: ", abs(grid_search.best_score_))

# # predict on the test data using the best model
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)

# eval_y_test, eval_y_pred = np.exp(y_test), np.exp(y_pred)

# mse = sklearn.metrics.mean_squared_error(eval_y_test, eval_y_pred)
# rmse = sklearn.metrics.mean_squared_error(eval_y_test, eval_y_pred, squared=False)
# mae = sklearn.metrics.mean_absolute_error(eval_y_test, eval_y_pred)
# calculate_mape = sklearn.metrics.mean_absolute_percentage_error(
#     eval_y_test, eval_y_pred
# )

# print("\nMSE:", mse, "\nRMSE:", rmse, "\nMAE:", mae, "\nMAPE:", calculate_mape)

In [ ]:
# Done: wziąc te obsadzenia stanów które nie są jedynkami ani zerami
# nie pomogły
# Done: Osobne modele na danych elektornicznych i danych fizycznych
# same fizyczne dane straciły mniej niż procent na testowym i 3 na treningowym
# same elektoniczne starciły 8% na testowym, ale to w sumie 3 ficzery + obsadzenia stanów

# shap <- do istotnosci ficzerów
# DScribe:
# Edwald sum
# MBTR

#